In [2]:
# 파일 데이터를 처리하기 위해 사용되는 모듈
import os

In [3]:
# 시간 데이터를 처리하기 위해 사용되는 모듈
import time
# 데이터 분석 pandas 모듈
import pandas as pd

In [4]:
# selenium 사용
from selenium.webdriver.common.by import By # 태그 존재 여부 확인 가능
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리

In [5]:
# beautifulsoup4 사용
from bs4 import BeautifulSoup
# selenium은 가상 브라우저를 활용하여 크롤링을 진행
from selenium import webdriver

In [6]:
# 드롭 다운 메뉴값을 선택할 수 있는 라이브러리
from selenium.webdriver.support.ui import Select

In [7]:
# 엑셀 함수 작업 자동화
import openpyxl as op
import requests

In [8]:
# 항상 최신버전의 chromedriver를 자동으로 사용할 수 있도록 하는 라이브러리
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# 크롤링한 이미지를 저장하기 위해 사용되는 모듈
import urllib.request

In [193]:
# 버전 업데이트 필요없이 항상 최신 버전의 구글 드라이브를 사용할 수 있도록 함
driver = webdriver.Chrome(ChromeDriverManager().install())
# 드라이브 열리는 동안 기다림
driver.implicitly_wait(3)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST driver version for 104.0.5112
[WDM] - Get LATEST driver version for 104.0.5112
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\shinyumi\.wdm\drivers\chromedriver\win32\104.0.5112.79]
<ipython-input-193-841b129e71dc>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [194]:
# 위 코드 실행하고, 창을 닫지 않아야함
# 접근할 주소, 레진코믹스 연령 제한 필터 해제 페이지
page_address='https://www.lezhin.com/ko/drama?page=0&sub_tags=all'
# 암묵적으로 모든 자원이 로드될 때까지 기다리게 하는 시간
driver.implicitly_wait(3)
# # 크롤링할 사이트 호출, url에 접근하는 메소드
driver.get(page_address)

In [ ]:
# 작품 제목
/html/body/main/section/section/ul/li[1]/a/div[2]/div[1]
#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__info > div.lzComic__title

# 작품 작가
/html/body/main/section/section/ul/li[1]/a/div[2]/div[2]
#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__info > div.lzComic__artist

# 썸네일 이미지
/html/body/main/section/section/ul/li[1]/a/div[1]/picture/img
#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__thumb > picture > img

# 세부 페이지(href)
/html/body/main/section/section/ul/li[1]/a
#exhibit-sub-tags > li:nth-child(1) > a

# 제목
#comic-info > div > h2

# 장르
#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__genre

# 연령 기준
#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__rating

# 연재 정보
#main > section.episode > header > p.episode__schedule

# 세부 페이지 내 작품 정보
/html/body/main/section[1]/div/button
#comic-info > div > button

# 작품 소개
#comic-info-extend > dialog > div.comicInfoExtend__synopsis > p

# 플랫폼 이름
레진코믹스

In [19]:
# 작품 제목
title=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__info > div.lzComic__title').text

# 작품 작가
writer=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__info > div.lzComic__artist').text

# 작품 썸네일
imgUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child(1) > a > div.lzComic__thumb > picture > img').get_attribute("src")

# 세부 페이지 링크
webtoonUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child(1) > a').get_attribute("href")

In [27]:
# 레진 코믹스의 특성 상 페이지가 넘어가면서 list가 리셋
elems=driver.find_elements(By.CSS_SELECTOR, '#exhibit-sub-tags > li')
len(elems)

36

In [23]:
# 실제로 활용하는 부분
col1='title'
col2='writer'
col3='imgUrl'
col4='webtoonUrl'
col_list=[col1, col2, col3, col4]
ta_df=pd.DataFrame(columns=col_list)
ta_df

,title,writer,imgUrl,webtoonUrl


In [24]:
while True:    
    # 상세 페이지로 들어가기 전에 제목, 작가, 썸네일 이미지 확보
    elems=driver.find_elements(By.CSS_SELECTOR, '#exhibit-sub-tags > li')

    # 목록 전체를 순차적으로 확인
    for i in range(0, len(elems)):
        # 작품 제목
        title=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__info > div.lzComic__title').text

        # 작품 작가
        writer=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__info > div.lzComic__artist').text

        # 작품 썸네일
        imgUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__thumb > picture > img').get_attribute("src")

        # 세부 페이지 링크
        webtoonUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a').get_attribute("href")
        
        # 임시 데이터프레임에 삽입
        df_tem={col1:title, col2:writer, col3:imgUrl, col4:webtoonUrl}
        
        # 전체 데이터프레임에 삽입
        ta_df=ta_df.append(df_tem,ignore_index=True)
        
        print(str(i+1) + " " + title + " 추출")
        
    # 다음 페이지가 더 존재하는지의 여부를 확인
    status=driver.find_element(By.XPATH, '/html/body/main/section/div[3]/button[2]').get_attribute("disabled")
    
    # 다음 페이지가 더 존재하지 않는다면, 중지
    if bool(status) is True:
        break;
        
    # 목록 넘기기
    driver.find_element(By.XPATH, '/html/body/main/section/div[3]/button[2]').click()
    time.sleep(2)

1 스머프 월드 추출
2 펑크 덩크 펑크 추출
3 열광 추출
4 야화첩 추출
5 칼과 꽃 추출
6 귀야곡 추출
7 히트앤런 추출
8 이삭의 양 추출
9 체험! XX의 현장 추출
10 알파 메이트 추출
11 너를 찾는 파도 추출
12 우연한 동거 추출
13 코스모 섹스 판타지 추출
14 마이크 없이도 들려 추출
15 조개소년 추출
16 빅앤빅 추출
17 개차반 추출
18 펌블 추출
19 [광공단편선] 한 번의 실수 추출
20 걷지 않는 다리 추출
21 그놈은 공이었다 추출
22 도깨비도 수풀이 있어야 모인다 추출
23 호랑이꽃 추출
24 데어, 낫 데어 추출
25 눈꽃 나으리 추출
26 인시크릿 추출
27 인연 추출
28 리버설 추출
29 리미티드 런 추출
30 매드 플레이스 추출
31 [광공단편선] 굿 헌터 & 배드 프레이 추출
32 [광공단편선] 거울의 경계 추출
33 언포기버블 추출
34 야쿠자가 사랑을 한다면 추출
35 이웃집 떡 추출
36 우주의 별 추출
1 구원하는 법 참 쉽습니다 추출
2 기물들의 세계 추출
3 요나와 창일 추출
4 장미와 샴페인 추출
5 러브 인 키치 추출
6 더블트랩 추출
7 3월의 보름을 조심하라 추출
8 에로 레볼루션 추출
9 페로몬 포비아 추출
10 블라인드 플레이 추출
11 후즈 유어 대디 추출
12 나의 수하 추출
13 남보다 못한 사이 추출
14 촉수 레시피 추출
15 러브 오어 헤이트 추출
16 홈보이(Homeboy) 추출
17 다크폴 추출
18 [광공단편선] Hellen (헬렌) 추출
19 검은 영혼의 무대 추출
20 그래도, 좋아해? 추출
21 가십(gossip) 추출
22 결혼하는 남자 [연재] 추출
23 삼자대면 추출
24 체크 메이트 [연재] 추출
25 [광공단편선] Blindspot 추출
26 심연 추출
27 신입사원 추출
28 애인셔틀 추출
29 키스 미, 라이어 (Kiss Me, Liar) 추출
30 킹스메이커 ~Triple Crown~ [완전판] [연재] 추출
31 리베타 추출
32 백만원의 로맨스 추

17 더 스캐터 [연재] 추출
18 도와줘요, 아저씨! 추출
19 신림과 봉천 사이 추출
20 짙은 [웹툰판][무삭제판] 추출
21 장인어른과 첫날밤을 보내버리면 [연재] 추출
22 @퍼피 러브 추출
23 너와의 거리 추출
24 탐닉 추출
25 보이즈 러브(Boys Love) 추출
26 나비, 호랑이를 물다 외전 [무삭제판] 추출
27 마스크 남자는 사랑하고 싶지 않은데 추출
28 연애 예행연습 추출
29 귀여운 유두에 첫눈에 반해 추출
30 단편선을 부탁해! 추출
31 꽃이 오르는 식탁 추출
32 수상한 동거 [연재] 추출
33 애신록:진 추출
34 젤러시 추출
35 방 건너 충성! 추출
36 가부키초 배드 트립 추출
1 고양이의 수트 개의 넥타이 추출
2 b씨만 몰라 추출
3 독점! 마이 히어로 시리즈 추출
4 힐링 패러독스 추출
5 짐승 추출
6 스윗 스팟 추출
7 어느 쪽이 좋아? 추출
8 내일은 어느 쪽! 추출
9 실연 중독 추출
10 반하는 약을 먹은 완벽남이 위험합니다! 추출
11 인 마이 클로젯 추출
12 연상연하 추출
13 옆집 남자가 뱀파이어 추출
14 연애의 공정성 추출
15 로맨티컬 비하인드 추출
16 조개소년[개정판] 추출
17 핫클립 추출
18 Teach Me! 추출
19 썩어빠진 댄디즈 추출
20 BL의 정석 추출
21 방탕아와 사랑의 구멍 추출
22 사내연애시뮬레이션 추출
23 보고싶어, 루시퍼 추출
24 울프 인더 하우스[개정판] 추출
25 무엇에 쓰는 물건인고? 추출
26 완전무결하게 사로잡히다 추출
27 일하고 싸우고 사랑하라(완전판) 추출
28 두근두근 사내연애 추출
29 투게더 더 시리즈(2gether the series) [연재] 추출
30 저기요, 황태자님! 추출
31 S to A (에스 투 에이) 추출
32 욕망의 일주일 ~오메가버스 단편선~ 추출
33 내겐 너무 자극적인 소꿉친구 추출
34 쿠로에와 에덴의 정원 추출
35 악벽 추출
36 나의 주인이 되어줘 추출
1 이 사랑의 성공 확률은? 추출
2 간극 추출
3 사

26 싫어하는 알파에게 안기다니 추출
27 백람의 짐승 추출
28 몽룡전 추출
29 브리프리프리 추출
30 나 냄새나? 추출
31 탐닉 [개정판] 추출
32 도쿄 남자 교토 남자 추출
33 허니 보이스♥소악마 베이비짱 추출
34 쿠로의 남자친구는 팬티에 열중?! 추출
35 위험한 천사와 최면에 빠진 건달 추출
36 키스하지 마 추출
1 형의 가장 맛있는 부분 추출
2 바른 망상 강좌 추출
3 일그러진 우리들의 모습 추출
4 메종 드 비치에 어서오세요! 추출
5 반야가인 [개정판] [연재] 추출
6 이상동몽 추출
7 이 내가 신음 따위를 낼 리 없어!! 추출
8 키스 앤드 나이트 추출
9 가면 너머에, 키스 추출
10 시집 안 갈 거야! 추출
11 로미오의 순정 추출
12 사랑하라 괴물군 추출
13 밀면 당기고 당기면 밀어라 추출
14 녹의 꿈 좌우 추출
15 믿기지는 않겠지만 추출
16 바리왕자 외전 [개정판] 추출
17 이츠키와 하루 추출
18 딜리버리 허그 테라피 추출
19 다크니스 하운드 -DARKNESS HOUND- 추출
20 커튼콜은 아직 이르다 추출
21 마이 와이프 데빌 [개정판] 추출
22 울보 군은 심술 악마의 손바닥 안 추출
23 고란이 전 [개정판] 추출
24 고지식한 형을 헐떡이게 하는 법 추출
25 운명의 접촉 [웹툰] 추출
26 야한 강아지의 성장 기록 추출
27 천재 소년 에자키의 상사병 추출
28 날 좀 싫어해줘 내 옆의 늑대 추출
29 오메가 휴가 삼삼구도 추출
30 사장님의 고양이 추출
31 꽃이 필 무렵, 사랑할 무렵 추출
32 나의 경찰관 추출
33 울보 샐러리맨의 사랑법 추출
34 나무꾼이 아니야 추출
35 양키 군은 타락하고 싶어 추출
36 하천부지 로미쥬리 항쟁곡 [개정판] 추출
1 최고의 악당 추출
2 블루 러스트 추출
3 Core Scramble (코어스크램블) 추출
4 쪽빛에 물들다 [연재] 추출
5 사랑은 AV 현장에서 시작된다 추출
6 후즈 더 라이어 추출
7 [비애] 세상에 나쁜 알파는 없다 추출
8 꽃은 토

19 천연 소설가에게 사랑을 가르쳐주게 되었습니다 추출
20 달콤한 피 추출
21 [비애] 왕자의 열에 녹아내릴 때까지 추출
22 흐르고 넘쳐서 참을 수 없어 추출
23 내가 너 따위를 좋아할 리 없어 추출
24 꽃을 달래는 햇살 추출
25 미완의 계절 [개정판] 추출
26 연애♥ 포르노 소설가 추출
27 사디스트 유령이 재워 주지 않는다 추출
28 노모어론리나이트 추출
29 밀명 추출
30 동아리 후배가 들이대고 있습니다 추출
31 나이트 밀크 헤븐 추출
32 슈거 드러그 추출
33 컨시어지의 연인 추출
34 ASMR로 가자 추출
35 [비애] 안 잡아먹어요? 늑대 씨 추출
36 사랑하는 알파의 양육법 추출
1 신령님은 독점하고 싶어! 추출
2 위즈덤즈의 동물 친구들 추출
3 페이드 [개정판] 추출
4 더블 스탠더드 추출
5 프리 키세스(Free Kisses) 추출
6 Nights[나이츠] 추출
7 룰루랄라 블루스 추출
8 붉은 테아트르 추출
9 [비애] 제복을 걸쳤으면 거짓말을 잘할 수 있었는데 추출
10 관능 작가의 하인 길들이기 추출
11 오만한 용사와 방구석폐인 마왕의 연애 공략술 추출
12 [비애] 제대로 돌보겠습니다! 추출
13 그래도 좋아합니다, 카사하라 씨 추출
14 BL 작가, 때때로 늑대 추출
15 동경하는 사람은 42세 창부입니다 추출
16 들려, 사랑의 초침 소리가 추출
17 BARBARITIES 바베리티스 추출
18 왕을 사랑한 신데렐라 추출
19 please, marry me!! 플리즈 메리미!! 추출
20 워커홀릭 베이비 추출
21 [비애] 소 귀에 경읽기 삼 년 추출
22 당신의 로멘스 추출
23 농익어 가는 살결 추출
24 환희의 초상 추출
25 백왕자와 흑왕자 추출
26 잠자는 코모리 ~네 냄새에 잠들게 해줘~ 추출
27 JOY 추출
28 손끝의 잠열 추출
29 사랑의 노예 추출
30 봄이 왔다 추출
31 구속하고 싶을 만큼 I LOVE YOU 추출
32 거짓말, 뱀파이어가 어디 있어? 추출
33 새하얀 오메가를 주웠습니다 추

34 메종 드 단아 추출
35 [비애] 스트로베리 콤배트 추출
36 미친 사랑을 부탁해 추출
1 유령 노예의 교육 추출
2 호진 선생님의 시원한 방과 후 추출
3 취향의 발견 추출
4 건방진 그 녀석 추출
5 노예 벨보이의 사육법을 알려드립니다. 추출
6 섹스 정키 보이 추출
7 Take on me! [연재] 추출
8 친애하는 진에게 추출
9 좋아하니까, 헤어지자 [로제] 추출
10 아빠가 좋아하는 사람 추출
11 도망치는 사랑을 잡아라 추출
12 [비애] 진면목 추출
13 저기, 안 만질 거야? 추출
14 한 쌍이 되어 줘, 주인님 추출
15 [비애] 브라콤 형의 은밀한 우울 추출
16 아우의 남편 추출
17 ××당하고 사랑도 당하고 뭐가 뭔지 추출
18 Loved Circus 추출
19 악마와 함께 춤을 [개정판] 추출
20 섹스리스 추출
21 타림 ~아무도 모르는 용의 이야기~ 추출
22 다녀왔어, 어서 와 -빛나는 나날- 추출
23 Berry VERY MUCH (베리베리머치) 추출
24 빼앗고 소유하고 추출
25 흉터에 입맞춤 추출
26 쉬이, 실례할 것 같아요 -쌍둥이 형제의 좌우 습격- 추출
27 후계자는 참지 않아 추출
28 중년선생님의 연하남 추출
29 기억의 실 추출
30 한밤중의 이방인 추출
31 [비애] 너는 이제 물지 못한다 추출
32 S주임 토미야 아라타가 추락하는 장소 추출
33 겁쟁이의 사랑 추출
34 언젠가 사랑이 될 때까지 [개정판] 추출
35 골판지 상자 안 고양이씨 시즌2 추출
36 숨기면 사랑 추출
1 절친에게 갑자기 고백받아 당황스럽습니다. 추출
2 [비애] 대형신입의 교육담당 -사내 연애 중- 추출
3 친구와 비밀의 동거생활 추출
4 불감증인 나는 울고 싶다 추출
5 너를 기다리는 터미널 추출
6 늑대 남친은 말을 진짜 안 들어 추출
7 짐승 시크에게 잡혀버린 보석 추출
8 M샐러리맨과 S날라리 추출
9 길몽 단편집 추출
10 희구지심 추출
11 내가 임신하려는 이유 추출
12 [비애] 종달새가 울면 잡아줘 추출
13 [

...

27 진동·전달·자극 추출
28 사랑하지만, 하지 않는 사이 [로제] 추출
29 기간한정 남친을 유혹하는 법 추출
30 누구 때문에 고민하는가? 추출
31 [비애] 무자비한 너는 언제나 옳다 추출
32 [비애] 사랑을 그리면 색이 피어난다 추출
33 애정은 얄미움에 숨는다 추출
34 우리집 아이가 신세 좀 지겠습니다 추출
35 그 남자가 흥분을 가라앉히는 방법 추출
36 트와일라잇 언더그라운드+ 추출
1 너에 대한 마음이 멈추지 않아 추출
2 고의는 아니지만 죄송합니다 추출
3 기한 만료 첫사랑 추출
4 사랑을 드려요 추출
5 왕자의 귀환 추출
6 첫사랑 카운터 어택 [개정판] 추출
7 남자사냥꾼 추출
8 나르 키스 추출
9 경국의 환관 추출
10 아침부터 저녁까지 추출
11 [비애] 변변치 못한 그와 나 추출
12 토마토한 수확 추출
13 극야(極夜) 추출
14 슈가베이브 추출
15 나랑 왜 안 해? 추출
16 반라 엉덩이를 다루는 법♂ 추출
17 아웃사이드 포르노그래프 추출
18 오타쿠와 소꿉친구 추출
19 이름 없는 남창 군 추출
20 청년 세입자 추출
21 무조건 항복론 추출
22 짐승 스위치 추출
23 심술부리지 말아줘 추출
24 어린이 박스 추출
25 고양이과 남친을 어르는 법 추출
26 백년결정목록 추출
27 욕망 렌즈 추출
28 에로 만화가와 어시스턴트 추출
29 망상가, 사랑하다 추출
30 소꿉친구의 플래그에는 기대할 수 없어! 추출
31 [비애] 음마 군은 일을 잘 못해 추출
32 음란한 불치병입니다 추출
33 손님과 가족이 되었습니다 추출
34 보스의 조건 [개정판] 추출
35 위태하게 추출
36 풀어진 몸 추출
1 형, 나한테 시집 와 추출
2 나의 귀여운 주정뱅이 추출
3 유리창 너머의 너에게 추출
4 내 후배의 품이 너무 넓어서 무서워 추출
5 맹목적인 호스트에게 지명당했습니다 추출
6 싫어하는 연인 추출
7 꿈 속의 마이 달링 추출
8 천재는 연애 중 추출
9 제왕의 음란한 속박 추출
10 럼스프링가의 정경 추출
11 들고양이는 사랑

15 최악의 룸셰어 시작했습니다 추출
16 Honey Maple more more sugar 추출
17 너에게 맺히는 별 추출
18 사랑의 시작 추출
19 나를 괴롭혀줘 추출
20 부장님의 어떤 비밀 추출
21 놀 줄 아는 그와 고지식한 나의 교육적 동거생활 추출
22 사장님과 애인군 (개정판) 추출
23 장난꾸러기의 정복론 추출
24 선생님으로 연습해도 돼? 추출
25 로맨틱 상등 추출
26 Sweet Our 1R 추출
27 쇼우 선생님의 야한 비밀 추출
28 불꽃은 이미 쏘아졌다 추출
29 시작은 이별 추출
30 여우는 달콤한 포도를 먹는다 추출
31 고동보다 강하고, 호흡보다 깊게 추출
32 허니베리의 짐승 추출
33 정말 내가 BL 영화의 주연?! 추출
34 체스와 라윈 추출
35 첫경험 추출
36 선생님과 애견 추출
1 불순♡교제 추출
2 안고 싶지 않은 이상적 남자 추출
3 어리광쟁이 나의 남동생 추출
4 교사가 AV 감독과 사랑해도 괜찮을까 추출
5 라스트 오메가버스 추출
6 음마에겐 내가 필요해! 추출
7 사랑과 확신의 초상 추출
8 허니 글라사쥬~집사, 시작했습니다~ 추출
9 사랑이라니, 나답지 않지만 추출
10 치킨 스토커 추출
11 두 사람만의 짝사랑 추출
12 섹스 앤드 월즈 엔드 추출
13 요리남의 비장의 맛 추출
14 그들은 XX가 아니라고 말한다 추출
15 스트레이트 회사원이지만 매일 남자 밑에서 울고 있습니다 추출
16 네 사랑은 이 몸의 것 추출
17 연애 열등생의 음란 계획 추출
18 잠든 고양이는 입맞춤으로 눈뜨지 않는다 추출
19 파인더의 고동 추출
20 향기 나는 짐승 추출
21 천천히 끌어 안아줘 추출
22 소문의 연예계 뒷사정 추출
23 지옥의 끝에서 [개정판] 추출
24 이노우에, 시작할까요? 추출
25 심술궂은 사람 추출
26 가벼운 그와 나의 30일 추출
27 호화맨션, 왕자님 포함 추출
28 여장 남자와 변태 담임 추출
29 노 컬러 추출
30 모모와 만지 추출
31 미열의 열매 추출
32 그가 남긴 연인 [개정

31 블랙 타이 [웹툰판] 추출
32 장밋빛 연구와 꽃을 먹는 너 추출
33 한밤중의 지배자 추출
34 캡슐 남자친구 추출
35 불청객 러브돌 추출
36 망상가의 사랑 추출
1 시오의 음란한 사육 추출
2 니노미야는 기분이 좋으면 울고 그런다 추출
3 길들여지고 싶은 고슴도치 추출
4 가시의 왕 추출
5 첫사랑 동맹 추출
6 사랑 꿈 연인 추출
7 꽃보다 동정 추출
8 이 이상은 무리예요! 추출
9 첫사랑 갤러리 추출
10 나의 말못할 증후군 추출
11 녹스(NOX) 추출
12 동남향 끝방의 사랑(개정판) 추출
13 미혹의 의자 추출
14 SLOW LIFE EVERY DAY 추출
15 여장 간호사의 로망은 변태 의사 선생님 추출
16 나에게 어울리는 굴욕을 추출
17 너를 독차지할거야 추출
18 노려보면 사랑 추출
19 도마 위의 잉어는 두 번 뛰어 오른다 추출
20 이상적인 연애의 조건 추출
21 변태 만화가의 몸은 기분 좋아 추출
22 I want to know you 추출
23 [비애] 사랑에 희롱당하다 추출
24 멍멍이 연애론 추출
25 음란한 짐승의 사랑법 추출
26 제멋대로지만 사랑스러워 추출
27 짝사랑이즘 추출
28 널 기다리는 앙상블 추출
29 이 몸의 신부가 돼라! 추출
30 넘버 콜 추출
31 피어스 홀은 막을 수 없어 추출
32 잇자국에 사랑 추출
33 꿈의 뒷이야기를 알려줘 추출
34 집사 따위 나의 노예지 추출
35 [비애] 사랑에 빠져 죽을 것 같아 추출
36 [비애] 첫사랑은 어려워 추출
1 그의 동생을 사랑하다 추출
2 그림동화 메르헨의 진실 추출
3 가로지나 세로지나 꽃은 핀다 [개정판] [연재] 추출
4 좋아한다고 말할래! 추출
5 그대여, 네 이웃을 사로잡아라 추출
6 바이바이, 왕따군 [로제] 추출
7 짹짹이가 짹 추출
8 유령이 들러붙었습니다 추출
9 육식남과 쾌감플레이 추출
10 게이게이 체인지! 지금은 바뀌어 있는 중 추출
11 재와 발라드 추출
12 진짜라고 말해줘 추출
13 발효 청년 추출
14 걷잡을 수 없이 추

36 MILK TEA 추출
1 개발부장의 비밀 업무 추출
2 흘러, 넘치다 추출
3 룸메이트의 음란한 연구 추출
4 꿈의 세계 뒤편의 사랑 추출
5 사랑스러운, 사람 추출
6 과장님, 결혼합시다!! 추출
7 1인자의 욕망 추출
8 깊은 곳, 보이지 않는 무언가 2 -절정 인비지블 플레이- 추출
9 러브 카페 모카 추출
10 [비애] 꽃 드세요 추출
11 반짝 반짝 드롭 라이프 추출
12 좋아져버렸어 추출
13 과장님의 비밀 추출
14 형이랑 같이 만들어보아요!★호모 100명이면 될까요? 추출
15 로맨스로는 부족해 추출
16 버려진 고양이를 키우는 법 -동물 맨션에 어서 오세요- 추출
17 사춘기의 수런거림 추출
18 고양이는 항상 변덕스러워 추출
19 갖고 싶은 것과 갖고 싶어 하는 것 추출
20 [비애] No.1 호스트가 지명했습니다 추출
21 구미호와 파계승 추출
22 내가 좋아, 걔가 좋아? 추출
23 안돼요 선생님 추출
24 지상으로부터 10센티 추출
25 특별수사전선♂여장형사 활달계 -밀착 24시- 추출
26 고양이처럼 귀여워하고 싶어! 추출
27 여장남자 플레이어에게 공략당했습니다 추출
28 꽃제물 추출
29 사랑에 목매는 인큐버스 추출
30 한결같은 그 아이의 욕구불만 추출
31 그의 애인 ~우리 속의 실낙~ 추출
32 그 손의 열기를 겹쳐 추출
33 3일간의 한정 남친 ~남국의 꽃의 유혹~ 추출
34 리리컬 머슬 엔카운트 추출
35 너의 냄새 추출
36 욕정 아쿠아리움 추출
1 로맨틱 빗치 추출
2 눈부신 사랑의 추출
3 나도 네가 없었다면 아마 추출
4 지터버그(JITTER BUG) 추출
5 [비애] 우리의 사랑을 너는 모른다 추출
6 나를 지키겠다고 맹세해 추출
7 [비애] 나의 귀여운 암살자 추출
8 소소한 연애 추출
9 백조소년 추출
10 VS학생회장 추출
11 미남인데 유령이시네요 추출
12 매니악 러버 추출
13 [스파크] 너에게 바치는 나의 기도 추출
14 외모만 좋으면 다 좋아 추출
15 킬러와 운명의 사랑 [로제] 추출
16 키

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B978B3+2193587]
	Ordinal0 [0x00B30681+1771137]
	Ordinal0 [0x00A441A8+803240]
	Ordinal0 [0x00A46BB4+814004]
	Ordinal0 [0x00A46A72+813682]
	Ordinal0 [0x00A473EA+816106]
	Ordinal0 [0x00A9E519+1172761]
	Ordinal0 [0x00A8D7AC+1103788]
	Ordinal0 [0x00A9DAE2+1170146]
	Ordinal0 [0x00A8D5C6+1103302]
	Ordinal0 [0x00A677E0+948192]
	Ordinal0 [0x00A686E6+952038]
	GetHandleVerifier [0x00E40CB2+2738370]
	GetHandleVerifier [0x00E321B8+2678216]
	GetHandleVerifier [0x00C217AA+512954]
	GetHandleVerifier [0x00C20856+509030]
	Ordinal0 [0x00B3743B+1799227]
	Ordinal0 [0x00B3BB68+1817448]
	Ordinal0 [0x00B3BC55+1817685]
	Ordinal0 [0x00B45230+1856048]
	BaseThreadInitThunk [0x75376739+25]
	RtlGetFullPathName_UEx [0x773F8FEF+1215]
	RtlGetFullPathName_UEx [0x773F8FBD+1165]


In [26]:
# 가져온 결과 imgUrl에는 blank 처리된 부분이 있었음
ta_df

,title,writer,imgUrl,webtoonUrl
0,스머프 월드,"제이제이, 오태",https://ccdn.lezhin.com/v2/comics/504726572112...,https://www.lezhin.com/ko/comic/smurf_world
1,펑크 덩크 펑크,장마,https://ccdn.lezhin.com/v2/comics/540690143674...,https://www.lezhin.com/ko/comic/punkdunkfunk
2,열광,오늘만 사는 형제,https://ccdn.lezhin.com/v2/comics/487965268698...,https://www.lezhin.com/ko/comic/enthusiasticity
3,야화첩,변덕,https://ccdn.lezhin.com/v2/comics/589986783127...,https://www.lezhin.com/ko/comic/paintings
4,칼과 꽃,snob,https://ccdn.lezhin.com/v2/comics/462696719142...,https://www.lezhin.com/ko/comic/knife_flower
...,...,...,...,...
4535,재난이 사랑이 되다,치바 료코,https://ccdn.lezhin.com/files/assets/blank.png,https://www.lezhin.com/ko/comic/disaster_love
4536,사랑에 빠진 코페르니쿠스,오이모 미오,https://ccdn.lezhin.com/v2/comics/477013880314...,https://www.lezhin.com/ko/comic/copernicus_in_...
4537,너는 두번째,나츠메 카츠라,https://ccdn.lezhin.com/v2/comics/451439404436...,https://www.lezhin.com/ko/comic/my_second_lover
4538,빼앗긴 시선,시시토 카라리,https://ccdn.lezhin.com/v2/comics/526828947138...,https://www.lezhin.com/ko/comic/the_eye_dwells


In [27]:
# 저장할 엑셀의 경로
excel_path='C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_BL_상세페이지.xlsx'
# 엑셀에 작성하기, 파일이름만 변경해서 저장
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
ta_df.to_excel(writer, index=False)
writer.save()

In [78]:
ta_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_BL_상세페이지.xlsx")

In [79]:
len(ta_df)

4540

In [80]:
ta_df

,title,writer,imgUrl,webtoonUrl
0,스머프 월드,"제이제이, 오태",https://ccdn.lezhin.com/v2/comics/504726572112...,https://www.lezhin.com/ko/comic/smurf_world
1,펑크 덩크 펑크,장마,https://ccdn.lezhin.com/v2/comics/540690143674...,https://www.lezhin.com/ko/comic/punkdunkfunk
2,열광,오늘만 사는 형제,https://ccdn.lezhin.com/v2/comics/487965268698...,https://www.lezhin.com/ko/comic/enthusiasticity
3,야화첩,변덕,https://ccdn.lezhin.com/v2/comics/589986783127...,https://www.lezhin.com/ko/comic/paintings
4,칼과 꽃,snob,https://ccdn.lezhin.com/v2/comics/462696719142...,https://www.lezhin.com/ko/comic/knife_flower
...,...,...,...,...
4535,재난이 사랑이 되다,치바 료코,https://ccdn.lezhin.com/files/assets/blank.png,https://www.lezhin.com/ko/comic/disaster_love
4536,사랑에 빠진 코페르니쿠스,오이모 미오,https://ccdn.lezhin.com/v2/comics/477013880314...,https://www.lezhin.com/ko/comic/copernicus_in_...
4537,너는 두번째,나츠메 카츠라,https://ccdn.lezhin.com/v2/comics/451439404436...,https://www.lezhin.com/ko/comic/my_second_lover
4538,빼앗긴 시선,시시토 카라리,https://ccdn.lezhin.com/v2/comics/526828947138...,https://www.lezhin.com/ko/comic/the_eye_dwells


In [81]:
col1='title' # 작품
col2='genre' # 장르
col3='ageCriteria' # 연령 기준 
col4='serialInformation' # 연재 정보
col5='serialDay' # 연재 요일
col6='imgUrl' # 썸네일 이미지
col7='introduce' # 작품 소개
col8='platform' # 플랫폼 정보
col_list=[col1, col2, col3, col4, col5, col6, col7, col8]
df_ta=pd.DataFrame(columns=col_list)
df_ta

,title,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform


In [107]:
ta_df['webtoonUrl'][119]

'https://www.lezhin.com/ko/comic/my_favorite_caught_me'

In [16]:
# 클릭이 되지 않는 오류를 해결
from selenium.webdriver.common.keys import Keys

In [56]:
page_address=ta_df['webtoonUrl'][2887]
page_address

'https://www.lezhin.com/ko/comic/comic_king'

In [68]:
from selenium.webdriver.common.action_chains import ActionChains

In [69]:
driver.find_element(By.CSS_SELECTOR, '#comic-info > div > button').send_keys(Keys.ENTER)

In [70]:
button = driver.find_element(By.XPATH, '/html/body/main/div/dialog/button')
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button).click(button).perform()

In [120]:
ta_df['webtoonUrl'][23] # 데어, 낫 데어

'https://www.lezhin.com/ko/comic/there_not_there_19'

In [128]:
driver.get(ta_df['webtoonUrl'][103]) # 판매종료된 서비스임에 유의

In [265]:
ta_df['webtoonUrl'][103]

'https://www.lezhin.com/ko/comic/maze_of_night'

In [261]:
for i in range(4518, len(ta_df)):
    # 작품 상세 페이지로 이동
    page_address=ta_df['webtoonUrl'][i]
    
    # 크롤링할 사이트 호출, url에 접근하는 메소드
    driver.get(page_address)
    
    # 암묵적으로 모든 자원이 로드될 때까지 기다리게 하는 시간
    driver.implicitly_wait(5)
    
    try:
        # GateWay 발생 여부 확인 시도, 작품 제목을 탐색
        driver.find_element(By.CSS_SELECTOR, '#comic-info > div > h2')
    except NoSuchElementException:
        # GateWay 발생 상황
        driver.back() # 다시 뒤로 돌아가기
        driver.implicitly_wait(10) # 자원 로드될 때까지 기다리기
        driver.get(page_address) # 다시 시도
        driver.implicitly_wait(10) # 자원 로드될 때까지 기다리기
    
    # 작품 제목
    title=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > h2').text
    
    # 작품 장르
    genre=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__genre').text

    # 연령 기준
    ageCriteria=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__rating').text

    # 연재 정보
    serialInformation=driver.find_element(By.CSS_SELECTOR, '#main > section.episode > header > p.episode__schedule').text
    
    # 연재 요일, 일단은 동일하게 추출하고, 정제 방향 고민
    serialDay=driver.find_element(By.CSS_SELECTOR, '#main > section.episode > header > p.episode__schedule').text

    # 썸네일 이미지
    imgUrl=driver.find_element(By.CSS_SELECTOR, '#comic-info > picture > img').get_attribute("src")
    
    # 작품 정보 페이지로 이동
    driver.find_element(By.CSS_SELECTOR, '#comic-info > div > button').send_keys(Keys.ENTER)
    driver.implicitly_wait(10)
    
    isCheck=True
    try:
        # 작품 소개가 존재하는지의 여부 탐색
        driver.find_element(By.CSS_SELECTOR,'#comic-info-extend > dialog > div.comicInfoExtend__synopsis')
    except NoSuchElementException:
        # 작품 소개가 존재하지 않음
        introduce=""
        isCheck=False
    
    if isCheck:
        introduce=driver.find_element(By.CSS_SELECTOR,'#comic-info-extend > dialog > div.comicInfoExtend__synopsis').text
        introduce=introduce.replace('작품소개\n', '') # 여러 개의 p 태그로 존재하므로 통째로 처리하고 정제
    
    # 작품 정보 페이지 닫기
    # button = driver.find_element(By.XPATH, '/html/body/main/div/dialog/button')
    # driver.implicitly_wait(15)
    # time.sleep(5)
    # ActionChains(driver).move_to_element(button).click(button).perform()
    driver.find_element(By.XPATH, '/html/body/main/div/dialog/button').send_keys(Keys.ENTER)
    # driver.implicitly_wait(10)
    # driver.find_element(By.XPATH, '/html/body/main/div/dialog/button').click()
    time.sleep(5)
    
    # 배열 형식으로 정리
    df_tem={col1:title, col2:genre, col3:ageCriteria, col4:serialInformation, col5:serialDay, col6:imgUrl, col7:introduce, col8:"레진코믹스"}
    
    # 통합 프레임에 병합
    df_ta=df_ta.append(df_tem, ignore_index=True)
    
    # 로그 작성
    print(str(i+1) + " " + title + " 추출")

4519 [비애] 수재 샐러리맨과 음흉한 여신 추출
4520 인기절정 왕자님의 연애 커맨드 추출
4521 동정 야쿠자와 출장 호스트 추출
4522 S·O·S· Salt or Sugar? 추출
4523 바보의 사랑 추출
4524 [웹툰판] 쾌감 스위트바디 추출
4525 2.5차원의 사랑 추출
4526 너무 싫다는 건 너무 좋아한다는 것! 추출
4527 곰 같은 남자에게 욕정 해버렸어 추출
4528 벚꽃 필 무렵에 추출
4529 좋아합니다, 절 봐주세요 추출
4530 벽 속의 천사 추출
4531 너와 나만의 이야기 추출
4532 이웃집 육식남♥ 추출
4533 거짓스러운 너의 입술 추출
4534 CANIS -Dear Hatter- 신장판 추출
4535 눈꺼풀 안의 여름 추출
4536 재난이 사랑이 되다 추출
4537 사랑에 빠진 코페르니쿠스 추출
4538 너는 두번째 추출
4539 빼앗긴 시선 추출
4540 롱 디스턴스 추출


In [262]:
df_ta

,title,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,스머프 월드,BL,19세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/504726572112...,제호그룹의 유력 후계자인 제규호는 사실 돈과 재미에만 관심이 있을 뿐.\n회사를 물...,레진코믹스
1,펑크 덩크 펑크,BL,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/540690143674...,호주에서 워킹홀리데이 중인 농장의 우수직원 이산들(23). 우연히 참석한 자선 파티...,레진코믹스
2,열광,BL,19세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/487965268698...,"불법 파이트 클럽의 선수였던, 최문혁. 소중한 동생들을 지키기 위해 무엇이든 하는 ...",레진코믹스
3,야화첩,BL,19세 이용가,매주 토요일 연재,매주 토요일 연재,https://ccdn.lezhin.com/v2/comics/589986783127...,"""개망나니 남색가""로 알려진 윤승호 도령이 우연히 접한 춘화집. 그림에 강한 매력을...",레진코믹스
4,칼과 꽃,BL,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/462696719142...,"달빛이 유난히 시리던 밤, 희량의 무자비한 칼끝에 연조의 가문은 몰락하고 만다. 하...",레진코믹스
...,...,...,...,...,...,...,...,...
4534,재난이 사랑이 되다,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/500607815240...,우는 아이도 뚝 그치게 하는(?) 야쿠자 카가미 라이.\n그의 인생은 한때 멀어졌었...,레진코믹스
4535,사랑에 빠진 코페르니쿠스,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/477013880314...,경영학과의 쇼고는 매서운 인상이라 친구를 사귀기 쉽지 않다. 하지만 사실은 성실하고...,레진코믹스
4536,너는 두번째,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/451439404436...,"불량학생 출신인 카도쿠라 마후유는 동생을 무지 사랑하는 형♥\n그러던 어느 날,사랑...",레진코믹스
4537,빼앗긴 시선,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/526828947138...,성적은 우수하지만 라이트 노벨을 과하게 좋아해 '교복이 흰색'이라는 웃기지도 않는 ...,레진코믹스


In [263]:
len(df_ta)

4539

In [264]:
# 저장할 엑셀의 경로
excel_path='C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_BL.xlsx'
# 엑셀에 작성하기, 파일이름만 변경해서 저장
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
df_ta.to_excel(writer, index=False)
writer.save()

In [90]:
# 중간에 갑자기 gateway time-out이 발생하는 경우 다시 driver.back() 하고 재추출 진행
#gateway=driver.find_element(By.CSS_SELECTOR, "body > center > h1").text
#if(gateway == '504 Gateway Time-out'):
    #driver.back()
/html/body/main/h2

In [111]:
from selenium.common.exceptions import NoSuchElementException

In [114]:
try:
    # Gateway 발생
    driver.find_element(By.CSS_SELECTOR, "body > center > h1"); 
    print(True)
except NoSuchElementException:
    # Gateway 발생하지 않은 경우
    print(False)

True


In [76]:
len(df_ta)

3414

In [ ]:
# 상세 페이지랑 467번 부분 일치 여부 확인 후에 재정제

In [58]:
# 세부 페이지에서 추출할 코드 구성
# 작품 제목
title=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > h2').text
# 작품 장르
genre=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__genre').text
# 연령 기준
ageCriteria=driver.find_element(By.CSS_SELECTOR, '#comic-info > div > div.comicInfo__genreGroup > span.comicInfo__rating').text
# 연재 정보
serialInformation=driver.find_element(By.CSS_SELECTOR, '#main > section.episode > header > p.episode__schedule').text
# 연재 요일
serialDay=driver.find_element(By.CSS_SELECTOR, '#main > section.episode > header > p.episode__schedule').text
# 썸네일 이미지
imgUrl=driver.find_element(By.CSS_SELECTOR, '#comic-info > picture > img').get_attribute("src")

'매주 금요일 연재'

In [61]:
# 작품 정보 페이지로 이동
driver.find_element(By.CSS_SELECTOR, '#comic-info > div > button').click()
time.sleep(2)

# 작품 소개
introduce=driver.find_element(By.CSS_SELECTOR,'#comic-info-extend > dialog > div.comicInfoExtend__synopsis > p').text

# 작품 정보 페이지 닫기
driver.find_element(By.CSS_SELECTOR, '#comic-info-extend > dialog > button').click()
time.sleep(2)

In [ ]:
# 상세 페이지로 들어가기 전에 제목, 작가, 썸네일 이미지 확보
elems=driver.find_elements(By.CSS_SELECTOR, '#exhibit-sub-tags > li')

# 목록 전체를 순차적으로 확인
for i in range(0, len(elems)):
    # 작품 제목
    title=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__info > div.lzComic__title').text

    # 작품 작가
    writer=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__info > div.lzComic__artist').text

    # 작품 썸네일
    imgUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a > div.lzComic__thumb > picture > img').get_attribute("src")

    # 세부 페이지 링크
    webtoonUrl=driver.find_element(By.CSS_SELECTOR, '#exhibit-sub-tags > li:nth-child('+str(i+1)+') > a').get_attribute("href")

In [292]:
# 엑셀 병합 단계 진행
detail_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_소년_상세페이지.xlsx")
detail_df

,title,writer,imgUrl,webtoonUrl
0,던전 속 사정,레바,https://ccdn.lezhin.com/v2/comics/534750346018...,https://www.lezhin.com/ko/comic/dungeon
1,4컷용사,고지라군,https://ccdn.lezhin.com/v2/comics/248/images/t...,https://www.lezhin.com/ko/comic/cartoon_hero
2,애늙은이,환댕,https://ccdn.lezhin.com/v2/comics/621722092385...,https://www.lezhin.com/ko/comic/gilgamesh
3,천지해,핑푸,https://ccdn.lezhin.com/v2/comics/59/images/ta...,https://www.lezhin.com/ko/comic/cheonjihae
4,박씨유대기,슬라정,https://ccdn.lezhin.com/v2/comics/672740585766...,https://www.lezhin.com/ko/comic/waitmrpark
...,...,...,...,...
1087,아키라 - 그리프터 자칭의협사기사,코세이 미쯔모리,https://ccdn.lezhin.com/v2/comics/776/images/t...,https://www.lezhin.com/ko/comic/akira
1088,ARTHUMAN,딩졸,https://ccdn.lezhin.com/v2/comics/467491231288...,https://www.lezhin.com/ko/comic/arthuman
1089,등뒤의 사람,하현수,https://ccdn.lezhin.com/v2/comics/538975754282...,https://www.lezhin.com/ko/comic/thebackofpeople
1090,똑똑한 팬지군!,그림보이,https://ccdn.lezhin.com/v2/comics/494497301135...,https://www.lezhin.com/ko/comic/smartychimp


In [293]:
detail_df=detail_df.drop(labels=['imgUrl', 'webtoonUrl'], axis=1)
detail_df

,title,writer
0,던전 속 사정,레바
1,4컷용사,고지라군
2,애늙은이,환댕
3,천지해,핑푸
4,박씨유대기,슬라정
...,...,...
1087,아키라 - 그리프터 자칭의협사기사,코세이 미쯔모리
1088,ARTHUMAN,딩졸
1089,등뒤의 사람,하현수
1090,똑똑한 팬지군!,그림보이


In [294]:
info_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_소년.xlsx")
info_df

,title,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,던전 속 사정,판타지,19세 이용가,"매월 1일, 11일, 21일 연재","매월 1일, 11일, 21일 연재",https://ccdn.lezhin.com/v2/comics/534750346018...,마물을 물리치고 얻게 되는 전리품. 그 용도의 새로운(?) 해석으로 인해 재구축된 ...,레진코믹스
1,4컷용사,판타지,15세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/248/images/t...,마왕을 해치운 용사는 드디어 공주를 구하고 새 삶을 찾는다. 하지만 공주와의 달콤한...,레진코믹스
2,애늙은이,판타지,15세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/621722092385...,견습기사 트루디아는 넘치는 혈기에 그만 기사단에서 쫓겨나고 만다. 갈 곳을 찾아 방...,레진코믹스
3,천지해,판타지,15세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/59/images/ta...,"천계에 올바른 질서를 가져오기 위해 초대 옥황상제가 12지, 십장생, 오방신의 대표...",레진코믹스
4,박씨유대기,개그,15세 이용가,"매월 7일, 17일, 27일 연재","매월 7일, 17일, 27일 연재",https://ccdn.lezhin.com/v2/comics/672740585766...,평범한 남학생 박남주는 같은 반의 큐트한 여학생 박여주에게 끌린다. 하지만 그녀의 ...,레진코믹스
...,...,...,...,...,...,...,...,...
1087,아키라 - 그리프터 자칭의협사기사,액션,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/776/images/t...,악덕 사기꾼을 처단하는 천성의 감각 소유자 그리프터들의 통쾌한 서스펜스 스토리!\n...,레진코믹스
1088,ARTHUMAN,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/467491231288...,"상상력 에너지인 ‘아이마(IMA)’와, 상상을 구체화 시키는 기계 ‘크리에이터(Cr...",레진코믹스
1089,등뒤의 사람,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/538975754282...,무당인 할머니의 피를 이어받아 요괴의 기를 느끼며 볼 수 있는 고등학생 임시윤. 그...,레진코믹스
1090,똑똑한 팬지군!,개그,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/494497301135...,실험으로 사람보다 더 똑똑해진 침팬지 팬지군이 연구소를 탈출하여 벌어지는 유쾌하고 ...,레진코믹스


In [295]:
info_df=info_df.drop(labels=['title'], axis=1)
info_df

,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,판타지,19세 이용가,"매월 1일, 11일, 21일 연재","매월 1일, 11일, 21일 연재",https://ccdn.lezhin.com/v2/comics/534750346018...,마물을 물리치고 얻게 되는 전리품. 그 용도의 새로운(?) 해석으로 인해 재구축된 ...,레진코믹스
1,판타지,15세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/248/images/t...,마왕을 해치운 용사는 드디어 공주를 구하고 새 삶을 찾는다. 하지만 공주와의 달콤한...,레진코믹스
2,판타지,15세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/621722092385...,견습기사 트루디아는 넘치는 혈기에 그만 기사단에서 쫓겨나고 만다. 갈 곳을 찾아 방...,레진코믹스
3,판타지,15세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/59/images/ta...,"천계에 올바른 질서를 가져오기 위해 초대 옥황상제가 12지, 십장생, 오방신의 대표...",레진코믹스
4,개그,15세 이용가,"매월 7일, 17일, 27일 연재","매월 7일, 17일, 27일 연재",https://ccdn.lezhin.com/v2/comics/672740585766...,평범한 남학생 박남주는 같은 반의 큐트한 여학생 박여주에게 끌린다. 하지만 그녀의 ...,레진코믹스
...,...,...,...,...,...,...,...
1087,액션,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/776/images/t...,악덕 사기꾼을 처단하는 천성의 감각 소유자 그리프터들의 통쾌한 서스펜스 스토리!\n...,레진코믹스
1088,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/467491231288...,"상상력 에너지인 ‘아이마(IMA)’와, 상상을 구체화 시키는 기계 ‘크리에이터(Cr...",레진코믹스
1089,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/538975754282...,무당인 할머니의 피를 이어받아 요괴의 기를 느끼며 볼 수 있는 고등학생 임시윤. 그...,레진코믹스
1090,개그,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/494497301135...,실험으로 사람보다 더 똑똑해진 침팬지 팬지군이 연구소를 탈출하여 벌어지는 유쾌하고 ...,레진코믹스


In [296]:
result=pd.concat([detail_df, info_df], axis=1)
result

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,던전 속 사정,레바,판타지,19세 이용가,"매월 1일, 11일, 21일 연재","매월 1일, 11일, 21일 연재",https://ccdn.lezhin.com/v2/comics/534750346018...,마물을 물리치고 얻게 되는 전리품. 그 용도의 새로운(?) 해석으로 인해 재구축된 ...,레진코믹스
1,4컷용사,고지라군,판타지,15세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/248/images/t...,마왕을 해치운 용사는 드디어 공주를 구하고 새 삶을 찾는다. 하지만 공주와의 달콤한...,레진코믹스
2,애늙은이,환댕,판타지,15세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/621722092385...,견습기사 트루디아는 넘치는 혈기에 그만 기사단에서 쫓겨나고 만다. 갈 곳을 찾아 방...,레진코믹스
3,천지해,핑푸,판타지,15세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/59/images/ta...,"천계에 올바른 질서를 가져오기 위해 초대 옥황상제가 12지, 십장생, 오방신의 대표...",레진코믹스
4,박씨유대기,슬라정,개그,15세 이용가,"매월 7일, 17일, 27일 연재","매월 7일, 17일, 27일 연재",https://ccdn.lezhin.com/v2/comics/672740585766...,평범한 남학생 박남주는 같은 반의 큐트한 여학생 박여주에게 끌린다. 하지만 그녀의 ...,레진코믹스
...,...,...,...,...,...,...,...,...,...
1087,아키라 - 그리프터 자칭의협사기사,코세이 미쯔모리,액션,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/776/images/t...,악덕 사기꾼을 처단하는 천성의 감각 소유자 그리프터들의 통쾌한 서스펜스 스토리!\n...,레진코믹스
1088,ARTHUMAN,딩졸,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/467491231288...,"상상력 에너지인 ‘아이마(IMA)’와, 상상을 구체화 시키는 기계 ‘크리에이터(Cr...",레진코믹스
1089,등뒤의 사람,하현수,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/538975754282...,무당인 할머니의 피를 이어받아 요괴의 기를 느끼며 볼 수 있는 고등학생 임시윤. 그...,레진코믹스
1090,똑똑한 팬지군!,그림보이,개그,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/494497301135...,실험으로 사람보다 더 똑똑해진 침팬지 팬지군이 연구소를 탈출하여 벌어지는 유쾌하고 ...,레진코믹스


In [297]:
excel_path='C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_소년_최종.xlsx'
# 엑셀에 작성하기, 파일이름만 변경해서 저장
writer=pd.ExcelWriter(excel_path, engine='xlsxwriter')
result.to_excel(writer, index=False)
writer.save()

In [298]:
bl_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_BL_최종.xlsx")
bl_df

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,스머프 월드,"제이제이, 오태",BL,19세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/504726572112...,제호그룹의 유력 후계자인 제규호는 사실 돈과 재미에만 관심이 있을 뿐.\n회사를 물...,레진코믹스
1,펑크 덩크 펑크,장마,BL,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/540690143674...,호주에서 워킹홀리데이 중인 농장의 우수직원 이산들(23). 우연히 참석한 자선 파티...,레진코믹스
2,열광,오늘만 사는 형제,BL,19세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/487965268698...,"불법 파이트 클럽의 선수였던, 최문혁. 소중한 동생들을 지키기 위해 무엇이든 하는 ...",레진코믹스
3,야화첩,변덕,BL,19세 이용가,매주 토요일 연재,매주 토요일 연재,https://ccdn.lezhin.com/v2/comics/589986783127...,"""개망나니 남색가""로 알려진 윤승호 도령이 우연히 접한 춘화집. 그림에 강한 매력을...",레진코믹스
4,칼과 꽃,snob,BL,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/462696719142...,"달빛이 유난히 시리던 밤, 희량의 무자비한 칼끝에 연조의 가문은 몰락하고 만다. 하...",레진코믹스
...,...,...,...,...,...,...,...,...,...
4534,재난이 사랑이 되다,치바 료코,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/500607815240...,우는 아이도 뚝 그치게 하는(?) 야쿠자 카가미 라이.\n그의 인생은 한때 멀어졌었...,레진코믹스
4535,사랑에 빠진 코페르니쿠스,오이모 미오,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/477013880314...,경영학과의 쇼고는 매서운 인상이라 친구를 사귀기 쉽지 않다. 하지만 사실은 성실하고...,레진코믹스
4536,너는 두번째,나츠메 카츠라,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/451439404436...,"불량학생 출신인 카도쿠라 마후유는 동생을 무지 사랑하는 형♥\n그러던 어느 날,사랑...",레진코믹스
4537,빼앗긴 시선,시시토 카라리,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/526828947138...,성적은 우수하지만 라이트 노벨을 과하게 좋아해 '교복이 흰색'이라는 웃기지도 않는 ...,레진코믹스


In [299]:
drama_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_드라마_최종.xlsx")
drama_df

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,하고 싶게,오계,드라마,19세 이용가,"매월 1일, 11일, 21일 연재","매월 1일, 11일, 21일 연재",https://ccdn.lezhin.com/v2/comics/477670975897...,왜 그들 시선에 제 몸이 떨리는 걸까요? 제가 그들을 '하고 싶게' 만드는 걸까요?...,레진코믹스
1,한지붕 아래,"그레고르, 짱이",드라마,19세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/574049135132...,"중졸, 무자본, 무스펙... 그리고 고아\n이런 나에게 온 유일한 한줄기 빛은......",레진코믹스
2,아씨,"캄피스, J&B",드라마,19세 이용가,매주 토요일 연재,매주 토요일 연재,https://ccdn.lezhin.com/v2/comics/533772649817...,"13살, 막내 아씨를 보기만 해도 얼굴 붉히던 돌쇠는 어느새 건장한 청년으로 자란다...",레진코믹스
3,플라잉 하이,"마들렌, 루붕",드라마,19세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/639954934962...,당장 내일의 알바 자리가 사랑보다 중요한 `정혁`\n등록금부터 생활비까지...연애는...,레진코믹스
4,불륜학개론,"Jiro, 쫑이",드라마,19세 이용가,매주 금요일 연재,매주 금요일 연재,https://ccdn.lezhin.com/v2/comics/579437418230...,"완벽한 와이프, 완벽한 직장... 이보다 더 완벽한 삶은 없을 것만 같았다.\n하지...",레진코믹스
...,...,...,...,...,...,...,...,...,...
3409,꿈을 안고 내일로,오타니 히로코,드라마,15세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/475599183989...,「야마시나 펜션」 시리즈 일곱 번째 이야기!!\n친구로부터 아이를 키우게 된\n와카...,레진코믹스
3410,그리워서,오타니 히로코,드라마,15세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/485819245107...,"홋카이도 교외에 위치한,\n자연과 사람이 만나는 펜션 ‘야마시나’!\n삶에 지친 사...",레진코믹스
3411,세 명의 신부와 돌아가며 XX,니와토리,드라마,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/661393518814...,"나에게 반한 아이돌 소녀, 행동도 가슴도 큰 전 여자친구, 한결같은 츤데레 날라리…...",레진코믹스
3412,어디까지 가봤어?,비비,일상,19세 이용가,매주 토요일 연재,매주 토요일 연재,https://ccdn.lezhin.com/v2/comics/580944669265...,비비가 처음 성욕이라는 걸 느끼고\n자위를 시작한 시점부터 자는 남편 옆에서 몰래 ...,레진코믹스


In [300]:
romance_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_로맨스_최종.xlsx")
romance_df

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,광안,"육미화, 소라, 라혜",로맨스,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/608525502834...,라혜 원작 <광안>의 웹툰화!\n묵묵하고 조용히 궁녀의 일을 하다가 지밀상궁이 되기...,레진코믹스
1,배드띵킹 다이어리,"호단, 랑라리",백합,19세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/488248695724...,"늘 함께였던 민지와 유나.\n열일곱 살 때부터 지금, 스물한 살까지 서로의 베스트 ...",레진코믹스
2,톡투미,은별,로맨스,19세 이용가,매주 월요일 연재,매주 월요일 연재,https://ccdn.lezhin.com/v2/comics/667065846071...,잘생기고 매너좋은 남자친구 세찬으로 행복한 나날을 보내는 래임. 하지만 사랑하는 그...,레진코믹스
3,내가 아니면 안 되는 주제에,마스미 소우,로맨스,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/515467535482...,"패턴사인 아키라는 타고난 정의감 때문에 후배를 감싸다가 사장을 때리고,\n회사는커녕...",레진코믹스
4,내 침대로 오는 남자들 [연재],"정해진, 앵, 김살구",로맨스,19세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/652817740418...,"사탕 같은 섹스 파트너들, 그리고 운명처럼 재회하게 된 십대 시절 나의 첫사랑.\n...",레진코믹스
...,...,...,...,...,...,...,...,...,...
2468,미소년 퀵! 서비스,김수영,로맨스,15세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/498024078770...,정말 맘에 드는 물건을 고르는 일만큼 어려운 일이 또 있을까?...\n귀엽고 착한 ...,레진코믹스
2469,아사의 강,노경해,로맨스,15세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/482468175360...,나 피가 분수처럼 솟는데도 아프기 보단 아름답다고 생각했어... 살아있는 몸 피가 ...,레진코믹스
2470,불건전 친구 [연재],"은랑호, 요모기모찌, 송보배",로맨스,15세 이용가,매주 토요일 연재,매주 토요일 연재,https://ccdn.lezhin.com/v2/comics/510180444163...,“원나잇 한 남자랑 친구 하는 미친X도 있니?”\n아역배우 시절 만나 20년 지기가...,레진코믹스
2471,연하의 선생님,천하,로맨스,전체 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/616170444167...,"외모, 성격, 재력 등 빼놓을 것 없이 완벽한 여고생 정나인.\n단 한 가지, 성적...",레진코믹스


In [301]:
boy_df=pd.read_excel("C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스_소년_최종.xlsx")
boy_df

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,던전 속 사정,레바,판타지,19세 이용가,"매월 1일, 11일, 21일 연재","매월 1일, 11일, 21일 연재",https://ccdn.lezhin.com/v2/comics/534750346018...,마물을 물리치고 얻게 되는 전리품. 그 용도의 새로운(?) 해석으로 인해 재구축된 ...,레진코믹스
1,4컷용사,고지라군,판타지,15세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/248/images/t...,마왕을 해치운 용사는 드디어 공주를 구하고 새 삶을 찾는다. 하지만 공주와의 달콤한...,레진코믹스
2,애늙은이,환댕,판타지,15세 이용가,완결,완결,https://ccdn.lezhin.com/v2/comics/621722092385...,견습기사 트루디아는 넘치는 혈기에 그만 기사단에서 쫓겨나고 만다. 갈 곳을 찾아 방...,레진코믹스
3,천지해,핑푸,판타지,15세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/59/images/ta...,"천계에 올바른 질서를 가져오기 위해 초대 옥황상제가 12지, 십장생, 오방신의 대표...",레진코믹스
4,박씨유대기,슬라정,개그,15세 이용가,"매월 7일, 17일, 27일 연재","매월 7일, 17일, 27일 연재",https://ccdn.lezhin.com/v2/comics/672740585766...,평범한 남학생 박남주는 같은 반의 큐트한 여학생 박여주에게 끌린다. 하지만 그녀의 ...,레진코믹스
...,...,...,...,...,...,...,...,...,...
1087,아키라 - 그리프터 자칭의협사기사,코세이 미쯔모리,액션,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/776/images/t...,악덕 사기꾼을 처단하는 천성의 감각 소유자 그리프터들의 통쾌한 서스펜스 스토리!\n...,레진코믹스
1088,ARTHUMAN,딩졸,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/467491231288...,"상상력 에너지인 ‘아이마(IMA)’와, 상상을 구체화 시키는 기계 ‘크리에이터(Cr...",레진코믹스
1089,등뒤의 사람,하현수,판타지,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/538975754282...,무당인 할머니의 피를 이어받아 요괴의 기를 느끼며 볼 수 있는 고등학생 임시윤. 그...,레진코믹스
1090,똑똑한 팬지군!,그림보이,개그,15세 이용가,NaN,NaN,https://ccdn.lezhin.com/v2/comics/494497301135...,실험으로 사람보다 더 똑똑해진 침팬지 팬지군이 연구소를 탈출하여 벌어지는 유쾌하고 ...,레진코믹스


In [302]:
len(boy_df)+len(romance_df)+len(drama_df)+len(bl_df)

11518

In [303]:
total_result=pd.concat([romance_df, boy_df, drama_df, bl_df], axis=0)
total_result

,title,writer,genre,ageCriteria,serialInformation,serialDay,imgUrl,introduce,platform
0,광안,"육미화, 소라, 라혜",로맨스,19세 이용가,매주 수요일 연재,매주 수요일 연재,https://ccdn.lezhin.com/v2/comics/608525502834...,라혜 원작 <광안>의 웹툰화!\n묵묵하고 조용히 궁녀의 일을 하다가 지밀상궁이 되기...,레진코믹스
1,배드띵킹 다이어리,"호단, 랑라리",백합,19세 이용가,매주 화요일 연재,매주 화요일 연재,https://ccdn.lezhin.com/v2/comics/488248695724...,"늘 함께였던 민지와 유나.\n열일곱 살 때부터 지금, 스물한 살까지 서로의 베스트 ...",레진코믹스
2,톡투미,은별,로맨스,19세 이용가,매주 월요일 연재,매주 월요일 연재,https://ccdn.lezhin.com/v2/comics/667065846071...,잘생기고 매너좋은 남자친구 세찬으로 행복한 나날을 보내는 래임. 하지만 사랑하는 그...,레진코믹스
3,내가 아니면 안 되는 주제에,마스미 소우,로맨스,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/515467535482...,"패턴사인 아키라는 타고난 정의감 때문에 후배를 감싸다가 사장을 때리고,\n회사는커녕...",레진코믹스
4,내 침대로 오는 남자들 [연재],"정해진, 앵, 김살구",로맨스,19세 이용가,매주 일요일 연재,매주 일요일 연재,https://ccdn.lezhin.com/v2/comics/652817740418...,"사탕 같은 섹스 파트너들, 그리고 운명처럼 재회하게 된 십대 시절 나의 첫사랑.\n...",레진코믹스
...,...,...,...,...,...,...,...,...,...
4534,재난이 사랑이 되다,치바 료코,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/500607815240...,우는 아이도 뚝 그치게 하는(?) 야쿠자 카가미 라이.\n그의 인생은 한때 멀어졌었...,레진코믹스
4535,사랑에 빠진 코페르니쿠스,오이모 미오,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/477013880314...,경영학과의 쇼고는 매서운 인상이라 친구를 사귀기 쉽지 않다. 하지만 사실은 성실하고...,레진코믹스
4536,너는 두번째,나츠메 카츠라,BL,19세 이용가,1화 무료 공개,1화 무료 공개,https://ccdn.lezhin.com/v2/comics/451439404436...,"불량학생 출신인 카도쿠라 마후유는 동생을 무지 사랑하는 형♥\n그러던 어느 날,사랑...",레진코믹스
4537,빼앗긴 시선,시시토 카라리,BL,19세 이용가,프롤로그 무료 공개,프롤로그 무료 공개,https://ccdn.lezhin.com/v2/comics/526828947138...,성적은 우수하지만 라이트 노벨을 과하게 좋아해 '교복이 흰색'이라는 웃기지도 않는 ...,레진코믹스


In [304]:
excel_path='C:\\Users\\shinyumi\\Desktop\\Excel\\레진코믹스.xlsx'
# 엑셀에 작성하기, 파일이름만 변경해서 저장
writer=pd.ExcelWriter(excel_path, engine='xlsxwriter')
total_result.to_excel(writer, index=False)
writer.save()